<a href="https://colab.research.google.com/github/anandsai410/Blockchain-technologies-Assignment-I/blob/main/ORT_Manual_Global_2903.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import nltk
import datetime
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk import punkt
import openpyxl
import pandas as pd
import numpy as np
import re
from datetime import datetime, date, time
import time
from dateutil.parser import parser
from dateutil.relativedelta import relativedelta
parser=parser()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!pip install spark-nlp
!pip install pyspark

import sparknlp
from sparknlp.annotator import DocumentAssembler, DateMatcher, MultiDateMatcher
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline

spark = sparknlp.start()
spark

In [205]:
df = pd.read_excel("GLobal_2903.xlsx")
df

,Manager,External Info,ORT manual,Assigned to,Number,State,Priority,Next SLA due date,Next SLA reason,Tags,Updated by,Customer contact
0,Marie Hardiman (I322855),2024-03-28 20:24:55 - Aaron Cooke (I565751) (E...,2024-04-02 22:30:00,Aaron Cooke (I565751),CS20240007061863,Awaiting Info,2 - High,2024-04-02 22:30:00,ORT (SLA),NaN,system,Swapna Shetty S (S0021840755)
1,Sourabh Shah (I033453),2024-03-29 05:24:59 - Carina Zimmermann (I8646...,2024-03-30 01:30:12,Abhijith Menon (I554905),CS20240007239407,In progress,2 - High,2024-03-30 01:30:12,ORT (SLA),NaN,system,Yogesh Yadav (S0024153773)
2,Joanna Monette Alas (I316191),2024-03-28 21:26:13 - Alfred Philip Esguerra (...,2024-04-05 04:31:00,Alfred Philip Esguerra (I562177),CS20240007233684,Awaiting Info,2 - High,2024-04-05 04:31:00,ORT (SLA),NaN,system,Jeffrey Mauro (S0007760160)
3,Joanna Monette Alas (I316191),2024-03-28 20:57:05 - Alfred Philip Esguerra (...,2024-04-04 12:32:06,Alfred Philip Esguerra (I562177),CS20240007283601,Awaiting Info,2 - High,2024-04-04 12:32:06,ORT (SLA),NaN,system,Priyesh shah (S0022754345)
4,Joanna Monette Alas (I316191),2024-03-29 00:49:32 - Alfred Philip Esguerra (...,2024-04-04 08:15:59,Alfred Philip Esguerra (I562177),CS20240007285907,In progress,3 - Medium,2024-04-04 08:15:59,ORT (SLA),NaN,system,Andy Chi (S0023348415)
...,...,...,...,...,...,...,...,...,...,...,...,...
445,Liam Hudson (I323795),2024-03-28 18:04:14 - Ciara Flynn (I529990) (E...,2024-04-01 05:29:56,Youssouf Sangare (I319380),CS20240006832810,In progress,3 - Medium,2024-04-01 05:29:56,ORT (SLA),#Q1_2024_Backlog_Tracking,I529990,Michelle McCann (S0025036898)
446,Liam Hudson (I323795),2024-03-26 04:33:49 - Youssouf Sangare (I31938...,2024-04-14 05:29:56,Youssouf Sangare (I319380),CS20240007116737,Awaiting Info,3 - Medium,2024-04-14 05:29:56,ORT (SLA),NaN,system,Andrea Johnson-Hall (S0022315314)
447,Liam Hudson (I323795),2024-03-26 04:06:38 - Youssouf Sangare (I31938...,2024-04-06 04:06:12,Youssouf Sangare (I319380),CS20240007147605,In progress,3 - Medium,2024-04-06 04:06:12,ORT (SLA),NaN,system,Michael Schröder (S0019596017)
448,Gabor Szilagyi (I524140),2024-03-28 16:38:57 - Zsolt Papp (I574643) (Ex...,2024-04-08 19:30:00,Zsolt Papp (I574643),CS20240007339316,In progress,3 - Medium,2024-04-08 19:30:00,ORT (SLA),NaN,system,Marc Lübeck (S0020026715)


In [206]:
ext_inf=df['External Info']
dte=df['ORT manual']
df['CP_ext_inf']=''
df['OA_ext_inf']=''
df['dates']=''
df['check']=''

In [207]:
pattern_SE = r'\<p(.*?)\</p>'
pattern_OA = r'\<br/>(.*?)\<br/>'
pattern_cde = r'\[code](.*?)\[/code]'

for i in range(0,len(df)):
  xstr=""
  ystr=""
  zstr=""
  x = re.findall(pattern_SE, df.iloc[i,1])
  y = re.findall(pattern_OA, df.iloc[i,1])
  z = re.findall(pattern_cde, df.iloc[i,1])
  for j in x:
     xstr = xstr+". "+str(j)
  for j in y:
     ystr = ystr+". "+str(j)
  for j in z:
     zstr = zstr+". "+str(j)
  df.iloc[i,12] = xstr
  df.iloc[i,12]=df.iloc[i,12]+"."+zstr
  df.iloc[i,13] = ystr

In [208]:
df

,Manager,External Info,ORT manual,Assigned to,Number,State,Priority,Next SLA due date,Next SLA reason,Tags,Updated by,Customer contact,CP_ext_inf,OA_ext_inf,dates,check
0,Marie Hardiman (I322855),2024-03-28 20:24:55 - Aaron Cooke (I565751) (E...,2024-04-02 22:30:00,Aaron Cooke (I565751),CS20240007061863,Awaiting Info,2 - High,2024-04-02 22:30:00,ORT (SLA),NaN,system,Swapna Shetty S (S0021840755),. >Hi there. >Engineering and OPS investigatio...,. You will receive an update about this incide...,,
1,Sourabh Shah (I033453),2024-03-29 05:24:59 - Carina Zimmermann (I8646...,2024-03-30 01:30:12,Abhijith Menon (I554905),CS20240007239407,In progress,2 - High,2024-03-30 01:30:12,ORT (SLA),NaN,system,Yogesh Yadav (S0024153773),". >Hello Yogesh,. >I am forwarding this case t...",. You will receive an update about this incide...,,
2,Joanna Monette Alas (I316191),2024-03-28 21:26:13 - Alfred Philip Esguerra (...,2024-04-05 04:31:00,Alfred Philip Esguerra (I562177),CS20240007233684,Awaiting Info,2 - High,2024-04-05 04:31:00,ORT (SLA),NaN,system,Jeffrey Mauro (S0007760160),". >Hi Jeffrey,<br /><br />Sorry to hear that t...",. You will receive an update about this incide...,,
3,Joanna Monette Alas (I316191),2024-03-28 20:57:05 - Alfred Philip Esguerra (...,2024-04-04 12:32:06,Alfred Philip Esguerra (I562177),CS20240007283601,Awaiting Info,2 - High,2024-04-04 12:32:06,ORT (SLA),NaN,system,Priyesh shah (S0022754345),". >Hi Priyesh,<br /><br />I trust you are well...",,,
4,Joanna Monette Alas (I316191),2024-03-29 00:49:32 - Alfred Philip Esguerra (...,2024-04-04 08:15:59,Alfred Philip Esguerra (I562177),CS20240007285907,In progress,3 - Medium,2024-04-04 08:15:59,ORT (SLA),NaN,system,Andy Chi (S0023348415),". >Hi Andy,<br /><br />Good day!<br /><br />I ...",,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,Liam Hudson (I323795),2024-03-28 18:04:14 - Ciara Flynn (I529990) (E...,2024-04-01 05:29:56,Youssouf Sangare (I319380),CS20240006832810,In progress,3 - Medium,2024-04-01 05:29:56,ORT (SLA),#Q1_2024_Backlog_Tracking,I529990,Michelle McCann (S0025036898),. ><b>Requester name:</b> Roy Wolgin<br /><b>R...,. . Contact phone or email: +1-. . . Contact p...,,
446,Liam Hudson (I323795),2024-03-26 04:33:49 - Youssouf Sangare (I31938...,2024-04-14 05:29:56,Youssouf Sangare (I319380),CS20240007116737,Awaiting Info,3 - Medium,2024-04-14 05:29:56,ORT (SLA),NaN,system,Andrea Johnson-Hall (S0022315314),". >Hi Andrea,. >We are reviewing your feedback...",. .,,
447,Liam Hudson (I323795),2024-03-26 04:06:38 - Youssouf Sangare (I31938...,2024-04-06 04:06:12,Youssouf Sangare (I319380),CS20240007147605,In progress,3 - Medium,2024-04-06 04:06:12,ORT (SLA),NaN,system,Michael Schröder (S0019596017),". >Hello Michael,. >we are reviewing your feed...",,,
448,Gabor Szilagyi (I524140),2024-03-28 16:38:57 - Zsolt Papp (I574643) (Ex...,2024-04-08 19:30:00,Zsolt Papp (I574643),CS20240007339316,In progress,3 - Medium,2024-04-08 19:30:00,ORT (SLA),NaN,system,Marc Lübeck (S0020026715),". >Hello Marc,. >Thank you for confirming thes...",,,


In [209]:
def is_date(string):
  if ((string[0:4]+string[5:7]+string[8:10]).isdigit()) or ((string[0:2]+string[3:5]+string[6:10]).isdigit()):
    return True
  else:
    return False

def convert_dte(x):
  if (x[0:2]+x[3:5]+x[6:10]).isdigit() and x[2]=="/" and int(x[0:2])<=12 and int(x[3:5])>12:
    y=datetime.strptime(x, "%m/%d/%Y")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:2]+x[3:5]+x[6:10]).isdigit() and x[2]=="." and int(x[0:2])<=12 and int(x[3:5])>12:
    y=datetime.strptime(x, "%m.%d.%Y")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:2]+x[3:5]+x[6:10]).isdigit() and x[2]=="-" and int(x[0:2])<=12 and int(x[3:5])>12:
    y=datetime.strptime(x, "%m-%d-%Y")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:4]+x[5:7]+x[8:10]).isdigit() and x[4]=="-":
    y=datetime.strptime(x, "%Y-%m-%d")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:2]+x[3:5]+x[6:10]).isdigit() and x[2]=="-":
    y=datetime.strptime(x, "%d-%m-%Y")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:4]+x[5:7]+x[8:10]).isdigit() and x[4]==".":
    y=datetime.strptime(x, "%Y.%m.%d")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:2]+x[3:5]+x[6:10]).isdigit() and x[2]==".":
    y=datetime.strptime(x, "%d.%m.%Y")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:4]+x[5:7]+x[8:10]).isdigit() and x[4]=="/":
    y=datetime.strptime(x, "%Y/%m/%d")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:2]+x[3:5]+x[6:10]).isdigit() and x[2]=="/":
    y=datetime.strptime(x, "%d/%m/%Y")
    y=y.strftime("%Y.%m.%d")
    return y
  else:
    return ''

In [210]:
# Step 1: Transforms raw texts to `document` annotation
document_assembler = (
    DocumentAssembler()
    .setInputCol("text")
    .setOutputCol("document")
)
# Step 2: Extracts one date information from text
date = (
    DateMatcher()
    .setInputCols("document") \
    .setOutputCol("date") \
    .setOutputFormat("yyyy.MM.dd")
)

pipl = Pipeline(stages=[document_assembler, date])

for i in range(0,len(df)):
  dtestr=""

  #ORT Assistant part
  text_oa = word_tokenize(df.iloc[i,13])
  for j in text_oa:
    if is_date(j):
      try:
        j=convert_dte(j)
      except (IndexError, ValueError):
        pass
      dtestr=dtestr+str(j)+","
#  df.iloc[i,14]+=dtestr+", "

  #Case Processor part
  text_cp = sent_tokenize(df.iloc[i,12])
  spark_df_cp = spark.createDataFrame(text_cp, StringType()).toDF("text")
  result_cp = pipl.fit(spark_df_cp).transform(spark_df_cp)
  resultdf_cp = result_cp.toPandas()
  for j in resultdf_cp['date']:
    if j != []:
      for k in j:
        dtestr=dtestr+str(k.result)+", "
  lst=[p.replace(" ","") for p in dtestr.split(",")]
  lst=list(set(lst))
  dtestr2=""
  for j in lst:
    if len(j)>5:
      dtestr2=dtestr2+j+","
  df.iloc[i,14]=dtestr2
  continue

In [211]:
df

,Manager,External Info,ORT manual,Assigned to,Number,State,Priority,Next SLA due date,Next SLA reason,Tags,Updated by,Customer contact,CP_ext_inf,OA_ext_inf,dates,check
0,Marie Hardiman (I322855),2024-03-28 20:24:55 - Aaron Cooke (I565751) (E...,2024-04-02 22:30:00,Aaron Cooke (I565751),CS20240007061863,Awaiting Info,2 - High,2024-04-02 22:30:00,ORT (SLA),NaN,system,Swapna Shetty S (S0021840755),. >Hi there. >Engineering and OPS investigatio...,. You will receive an update about this incide...,"2024.03.28,2024.02.15,2004.02.01,2024.03.30,20...",
1,Sourabh Shah (I033453),2024-03-29 05:24:59 - Carina Zimmermann (I8646...,2024-03-30 01:30:12,Abhijith Menon (I554905),CS20240007239407,In progress,2 - High,2024-03-30 01:30:12,ORT (SLA),NaN,system,Yogesh Yadav (S0024153773),". >Hello Yogesh,. >I am forwarding this case t...",. You will receive an update about this incide...,"2024.03.20,2024.03.28,2018.04.25,2024.03.30,20...",
2,Joanna Monette Alas (I316191),2024-03-28 21:26:13 - Alfred Philip Esguerra (...,2024-04-05 04:31:00,Alfred Philip Esguerra (I562177),CS20240007233684,Awaiting Info,2 - High,2024-04-05 04:31:00,ORT (SLA),NaN,system,Jeffrey Mauro (S0007760160),". >Hi Jeffrey,<br /><br />Sorry to hear that t...",. You will receive an update about this incide...,"2024.03.13,2024.03.14,",
3,Joanna Monette Alas (I316191),2024-03-28 20:57:05 - Alfred Philip Esguerra (...,2024-04-04 12:32:06,Alfred Philip Esguerra (I562177),CS20240007283601,Awaiting Info,2 - High,2024-04-04 12:32:06,ORT (SLA),NaN,system,Priyesh shah (S0022754345),". >Hi Priyesh,<br /><br />I trust you are well...",,"2399.01.11,2024.03.30,2024.01.11,",
4,Joanna Monette Alas (I316191),2024-03-29 00:49:32 - Alfred Philip Esguerra (...,2024-04-04 08:15:59,Alfred Philip Esguerra (I562177),CS20240007285907,In progress,3 - Medium,2024-04-04 08:15:59,ORT (SLA),NaN,system,Andy Chi (S0023348415),". >Hi Andy,<br /><br />Good day!<br /><br />I ...",,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,Liam Hudson (I323795),2024-03-28 18:04:14 - Ciara Flynn (I529990) (E...,2024-04-01 05:29:56,Youssouf Sangare (I319380),CS20240006832810,In progress,3 - Medium,2024-04-01 05:29:56,ORT (SLA),#Q1_2024_Backlog_Tracking,I529990,Michelle McCann (S0025036898),. ><b>Requester name:</b> Roy Wolgin<br /><b>R...,. . Contact phone or email: +1-. . . Contact p...,,
446,Liam Hudson (I323795),2024-03-26 04:33:49 - Youssouf Sangare (I31938...,2024-04-14 05:29:56,Youssouf Sangare (I319380),CS20240007116737,Awaiting Info,3 - Medium,2024-04-14 05:29:56,ORT (SLA),NaN,system,Andrea Johnson-Hall (S0022315314),". >Hi Andrea,. >We are reviewing your feedback...",. .,,
447,Liam Hudson (I323795),2024-03-26 04:06:38 - Youssouf Sangare (I31938...,2024-04-06 04:06:12,Youssouf Sangare (I319380),CS20240007147605,In progress,3 - Medium,2024-04-06 04:06:12,ORT (SLA),NaN,system,Michael Schröder (S0019596017),". >Hello Michael,. >we are reviewing your feed...",,"5959.01.09,2017.07.25,",
448,Gabor Szilagyi (I524140),2024-03-28 16:38:57 - Zsolt Papp (I574643) (Ex...,2024-04-08 19:30:00,Zsolt Papp (I574643),CS20240007339316,In progress,3 - Medium,2024-04-08 19:30:00,ORT (SLA),NaN,system,Marc Lübeck (S0020026715),". >Hello Marc,. >Thank you for confirming thes...",,"2024.04.08,",


In [212]:
count=0
for i in range(0,len(df)-1):
  omd = word_tokenize(str(df.iloc[i,2]))
  omdte = convert_dte(omd[0])
  if len(str(omdte))>5:
    for j in df.iloc[i,14].split(","):
      if str(omdte).replace(" ","")==str(j).replace(" ",""):
        print(df.iloc[i,4]+"   "+df.iloc[i,3]+"   "+df.iloc[i,0])
        df.iloc[i,15]='ORTM_OK'
        count=count+1
        continue
      else:
        continue

count2=0
for i in df['check']:
  if i=="ORTM_OK":
    count2=count2+1
print(count2)

CS20240007239407   Abhijith Menon (I554905)   Sourabh Shah (I033453)
CS20240007084147   Ananyasree S (I584960)   Sourabh Shah (I033453)
CS20240007046596   Benas Pakonis (I530140)   Liam Hudson (I323795)
CS20240007218746   Benas Pakonis (I530140)   Liam Hudson (I323795)
CS20230005739936   Benas Pakonis (I530140)   Liam Hudson (I323795)
CS20240007183332   Benas Pakonis (I530140)   Liam Hudson (I323795)
CS20240007337484   Benas Pakonis (I530140)   Liam Hudson (I323795)
CS20230005503656   Corey Edwards (I831722)   Chad Meyer (I857631)
CS20240007126450   Cristina Lahm Feron (I542852)   William Aquino Gibk (I862094)
CS20240006967311   Dave Dortants (I818243)   Chad Meyer (I857631)
CS20240007269818   David Rakoczi (I588638)   Gabor Szilagyi (I524140)
CS20240006964402   David Rakoczi (I588638)   Gabor Szilagyi (I524140)
CS20240007272163   David Rakoczi (I588638)   Gabor Szilagyi (I524140)
CS20240007345665   Deepak C S (I331119)   SHAMEER THAJUDEEN (I306331)
CS20240006820961   Deepak C S (I3311

In [199]:
for i in range(0,len(df)):
  if df.iloc[i,4]=="CS20240007284595":
    print(i)

206


In [ ]:
import openpyxl
df.to_excel('spark.xlsx', sheet_name='dates form spark')

In [203]:
text_cp = sent_tokenize(df.iloc[449,12])
spark_df_cp = spark.createDataFrame(text_cp, StringType()).toDF("text")
result_cp = pipl.fit(spark_df_cp).transform(spark_df_cp)
resultdf_cp = result_cp.toPandas()
#result_cp.show()
for j in resultdf_cp['date']:
  if j != []:
    for k in j:
      print(str(k.result))
      print(len(k.result))

h = word_tokenize(str(df.iloc[449,2]))
print()
print(len(convert_dte(h[0])))

2024.03.29
10
2024.04.08
10
2024.03.12
10
2024.03.29
10
2024.03.29
10
2024.03.29
10
2024.03.29
10

10


In [200]:
print(df.iloc[206,1])

2024-03-27 19:56:20 - Anna Nemeth (I589988) (External Info)
[code]<p style="margin: 0in; font-family: Calibri; font-size: 11.0pt;">Dear Team,</p>
<p style="margin: 0in; font-family: Calibri; font-size: 11.0pt;">&nbsp;</p>
<p style="margin: 0in; font-family: Calibri; font-size: 11.0pt;">I hope you are doing well.</p>
<p style="margin: 0in; font-family: Calibri; font-size: 11.0pt;">&nbsp;</p>
<p style="margin: 0in; font-family: Calibri; font-size: 11.0pt;">My name is Anna and I am handling Kristof's case while he is on leave.&nbsp;</p>
<p style="margin: 0in; font-family: Calibri; font-size: 11.0pt;">&nbsp;</p>
<p style="margin: 0in; font-family: Calibri; font-size: 11.0pt;">Our Engineering Team is still investigating your issue. We are monitoring the Incident and &nbsp;if there is any information from Engineering, we will let you know.</p>
<p style="margin: 0in; font-family: Calibri; font-size: 11.0pt;">&nbsp;</p>
<p style="margin: 0in; font-family: Calibri; font-size: 11.0pt; color: #16

In [136]:
for i in range(0,len(df)-1):
  z = re.findall(pattern_cde, df.iloc[i,1])
  for i in z:
    print(i)
    break

<p>Hi Jeffrey,<br /><br />Sorry to hear that the issue still persists.<br /><br />I'll let the team know and get back to you for an update.<br /><br />Regards,<br />Alfred</p>
<p>Hi Priyesh,<br /><br />I trust you are well.<br /><br />As the team did not see any error in the past few days, we have created an RCA request to check what caused the error.<br /><br />Further updates will be provided once information is available.<br /><br />Regards,<br />Alfred</p>
<p>Hi Andy,<br /><br />Good day!<br /><br />I have not received a substantial update yet from the team.<br /><br />Will keep you posted on their response.<br /><br />Regards,<br />Alfred</p>
<p>&nbsp; &nbsp;&nbsp;</p>
<p>Hello Mai Vu,<br /><br />Thank you for your patience on the ticket.<br />I will be creating an incident with the Analytics engineering team by EOD.<br />I will keep you posted on the same.<br /><br />Thanks &amp; Regards,<br />Ananya&nbsp;</p>
<p>Hello Arijit,&nbsp;<br /><br />I haven't gotten any further updates